In [ ]:

!pip install pandas dash

In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash import Dash
from dash.dependencies import Input, Output
import plotly.express as px

In [23]:
# Download and read the `spacex_launch_geo.csv`
#spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_csv_file = "spacex_launch_geo.csv"
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

In [26]:
# Create a dash application
app = dash.Dash(__name__)

app.layout = html.Div([    
    html.Header('My Launch Site'),    
     dcc.Dropdown(id='site-dropdown',
                     options=[
                             {'label': 'All Sites', 'value': 'ALL'},
                             {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                             {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                             {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                             {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                             ],
                     value='ALL',
                     placeholder='Select a Launch Site here',
                     searchable=True                    
                     ), 
    dcc.Graph(id = 'success-pie-chart') 
    ])
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        data  = spacex_df[spacex_df['class']==1]
       # data = filtered_df.groupby("Launch Site").count().reset_index()
        data = data.groupby("Launch Site").count().reset_index()        
        fig = px.pie(data, values='class', 
                        names= data['Launch Site'], 
                        title='All sites'
                    )
        return fig
    else:
        data = spacex_df[spacex_df['Launch Site']==entered_site]
        data = data.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig = px.pie(data, values='class count', 
                        names= data['class'], 
                        title= entered_site
                    )
        return fig
        # return the outcomes piechart for a selected site

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)

In [27]:
# Create a dash application
spacex_csv_file = "spacex_launch_geo.csv"
spacex_df=pd.read_csv(spacex_csv_file)

app = dash.Dash(__name__)
min_value= 100
max_value = 100000

app.layout = html.Div([    
    html.Header('My Launch Site'),    
     dcc.Dropdown(id='site-dropdown',
                     options=[
                             {'label': 'All Sites', 'value': 'ALL'},
                             {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                             {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                             {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                             {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                             ],
                     value='ALL',
                     placeholder='Select a Launch Site here',
                     searchable=True                    
                     ), 
      dcc.RangeSlider(id='payload-slider',
                    min=0, 
                    max=10000, 
                    step=1000,
                    marks={0: '0', 2000: '2000', 4000: '4000', 6000: '6000', 8000: '8000', 10000: '10000'},
                    value=[min_value, max_value]
                   ),
      dcc.Graph(id = 'success-payload-scatter-chart')  
    ])
# Function decorator to specify function input and output
@app.callback(
              Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property="value")
              )

def get_chart(entered_site, value):
    
    spacex_csv_file = "spacex_launch_geo.csv"
    spacex_df=pd.read_csv(spacex_csv_file)
    resultado = spacex_df[['Launch Site', 'Booster Version', 'Payload Mass (kg)', 'class']]
    max_payload = value[1]
    min_payload = value[0]
    if entered_site == 'ALL':        
       resultado = resultado[(resultado['Payload Mass (kg)']<max_payload) & (resultado['Payload Mass (kg)'] > min_payload)]
       fig = px.scatter(resultado, x="Payload Mass (kg)", y="class", color="Booster Version")
       return fig
    else:
        resultado = resultado[(resultado['Launch Site']==entered_site) & (resultado['Payload Mass (kg)']<max_payload)& (resultado['Payload Mass (kg)']>min_paylodas)]
        fig = px.scatter(resultado, x="Payload Mass (kg)", y="class", color="Booster Version")        
        return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8052)



Which site has the largest successful launches?
Answer: Its KSC LC-39A.

Which site has the highest launch success rate? 
Answer: Its SC LC-39A.

Which payload range(s) has the lowest launch success rate?
Answer: Its 0 to 6.000

Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?
Answer: Its F9n b4 b1041.1